In [3]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from z3 import *

# Carregando o dataset MNIST e selecionando apenas as classes 0 e 1
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(np.int8)

# Selecionar apenas as classes 0 e 1
mask = (y == 0) | (y == 1)
X = X[mask]
y = y[mask]

# Reduzir para as primeiras 1000 amostras (500 de cada classe, se possível)
num_samples = 500
X_class_0 = X[y == 0][:num_samples]
y_class_0 = y[y == 0][:num_samples]
X_class_1 = X[y == 1][:num_samples]
y_class_1 = y[y == 1][:num_samples]

X = np.vstack((X_class_0, X_class_1))
y = np.hstack((y_class_0, y_class_1))
feature_names = [f'pixel_{i}' for i in range(X.shape[1])]  # Nomes das características (pixels)

# Normalizando os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Treinando o modelo de regressão logística
model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200)
model.fit(X, y)

# Função para criar uma explicação minimal usando Z3 Solver
def minimal_explanation(model, instance, target_class, epsilon=1e-6, timeout=60000):
    num_features = instance.shape[0]
    weights = model.coef_
    intercepts = model.intercept_
    
    # Z3 Optimize
    opt = Optimize()
    opt.set("timeout", timeout)  # Adiciona limite de tempo (em milissegundos)
    
    # Variáveis Z3
    feature_selection = [Bool(f'f{i}') for i in range(num_features)]
    
    # Função de decisão do modelo
    def decision_function(weights, intercept, instance, selected_features):
        return Sum([If(selected_features[i], instance[i] * weights[i], 0) for i in range(num_features)]) + intercept
    
    # Adicionar restrições ao solver
    for i in range(len(model.classes_)):
        if i != target_class:
            decision_target = decision_function(weights[target_class], intercepts[target_class], instance, feature_selection)
            decision_other = decision_function(weights[i], intercepts[i], instance, feature_selection)
            constraint = decision_target > decision_other + epsilon
            opt.add(constraint)
    
    # Minimizar o número de características selecionadas
    opt.minimize(Sum([If(f, 1, 0) for f in feature_selection]))

    # Check satisfiability and get the model if possible
    result = opt.check()
    
    if result == sat:
        m = opt.model()
        explanation = [i for i in range(num_features) if m.evaluate(feature_selection[i])]
        explanation_features = [feature_names[i] for i in explanation]  # Nomes das características
        return result, explanation, explanation_features
    else:
        return result, None, None

# Exemplo de uso
instance = X[0]  # Exemplo de instância para explicar
target_class = model.predict([instance])[0]  # Classe prevista pelo modelo

result, explanation, explanation_features = minimal_explanation(model, instance, target_class)
if explanation:
    print("Objective: Minimize selected features")
    print("Solver result:", result)
    print("Model found. Explanation (indices):", explanation)
    print("Explicação minimal (nomes das características):", explanation_features)
else:
    print("No solution found")


C:\Users\gleilsonpedro\AppData\Local\Temp\ipykernel_19796\1089075391.py:20: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_class_0 = y[y == 0][:num_samples]
C:\Users\gleilsonpedro\AppData\Local\Temp\ipykernel_19796\1089075391.py:22: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_class_1 = y[y == 1][:num_samples]


IndexError: index 1 is out of bounds for axis 0 with size 1